# Colab Inferencing Notebook
The purpose of this file is to save and version control the Inferencing.ipynb file in Google Colab.  
The code in this file (and colab) will later be used for a .py file that will sent and processed in NRP (Nautilus).  
Colab's purpose is to debug this specific file and ensure that there are no errors before being sent to NRP since there is no debugging in NRP.